**Importation of libraries**

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import scipy.optimize as optimize
import requests
import io
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
#Constantes
date_format = '%d/%m/%Y'

**Importation of Yield Curve Euro + Cleaning**

In [4]:
DATAFLOW = "YC"
FREQ = "B"  # Daily - businessweek
REF_AREA = "U2"  # Euro area (changing composition)
CURRENCY = "EUR"
PROVIDER_FM = "4F"  # ECB
INSTRUMENT_FM = "G_N_A"  # Government bond, nominal, all issuers whose rating is triple A
PROVIDER_FM_ID = "SV_C_YM"  # Svensson model - continuous compounding - yield error minimisation
DATA_TYPE_FM = "SR_1M+SR_3M+SR_1Y+SR_5Y+SR_10Y+SR_30Y"

#N_OBS = 600
#PARAMETERS = f"lastNObservations={N_OBS}&detail=dat"
url = f"https://data-api.ecb.europa.eu/service/data/{DATAFLOW}/{FREQ}.{REF_AREA}.{CURRENCY}.{PROVIDER_FM}.{INSTRUMENT_FM}.{PROVIDER_FM_ID}.{DATA_TYPE_FM}"

response = requests.get(url, headers={'Accept': 'text/csv'})
response.raise_for_status()
df = pd.read_csv(io.StringIO(response.text), parse_dates=['TIME_PERIOD'], index_col=['TIME_PERIOD'])

In [5]:
df = df[["KEY", "OBS_VALUE"]].pivot(columns="KEY", values="OBS_VALUE").rename(columns = {"YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_30Y" : "30 Yr", "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_10Y" : "10 Yr",
                                                "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_5Y" : "5 Yr",
                                                "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_1Y" : "1 Yr",
                                                "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_3M" : "3 Mo"}).reindex(columns=["3 Mo", "1 Yr", "5 Yr", "10 Yr", "30 Yr"])

df.index = df.index.strftime(date_format)
df.index.names = ['Date']
df.reset_index

<bound method DataFrame.reset_index of KEY             3 Mo      1 Yr      5 Yr     10 Yr     30 Yr
Date                                                        
06/09/2004  2.034172  2.298838  3.457222  4.209220  4.988680
07/09/2004  2.040893  2.328891  3.479518  4.209626  4.975495
08/09/2004  2.044384  2.346666  3.507894  4.228419  4.978894
09/09/2004  2.037111  2.308988  3.430628  4.161872  4.946545
10/09/2004  2.034645  2.271566  3.374729  4.120981  4.918530
...              ...       ...       ...       ...       ...
22/11/2023  3.731569  3.398317  2.498392  2.585532  2.763743
23/11/2023  3.734965  3.411755  2.572756  2.665739  2.848310
24/11/2023  3.696261  3.423658  2.592966  2.686921  2.874095
27/11/2023  3.752325  3.400711  2.542137  2.628913  2.807179
28/11/2023  3.754842  3.342862  2.497847  2.589236  2.769308

[4919 rows x 5 columns]>

In [6]:
for x in ['3 Mo', '1 Yr', '5 Yr', '10 Yr', '30 Yr']:
    df[f'Return_YC_Euro_{x}'] = df[x] - df[x].shift(1)
df.dropna(inplace=True)

df.head(10)

KEY,3 Mo,1 Yr,5 Yr,10 Yr,30 Yr,Return_YC_Euro_3 Mo,Return_YC_Euro_1 Yr,Return_YC_Euro_5 Yr,Return_YC_Euro_10 Yr,Return_YC_Euro_30 Yr
Date,,,,,,,,,,
07/09/2004,2.040893,2.328891,3.479518,4.209626,4.975495,0.006721,0.030052,0.022295,0.000406,-0.013185
08/09/2004,2.044384,2.346666,3.507894,4.228419,4.978894,0.003491,0.017775,0.028376,0.018793,0.003399
09/09/2004,2.037111,2.308988,3.430628,4.161872,4.946545,-0.007273,-0.037677,-0.077266,-0.066547,-0.032349
10/09/2004,2.034645,2.271566,3.374729,4.120981,4.918530,-0.002466,-0.037423,-0.055899,-0.040891,-0.028015
13/09/2004,2.041363,2.307186,3.410319,4.138965,4.929555,0.006718,0.035620,0.035590,0.017984,0.011025
14/09/2004,2.037903,2.295860,3.395882,4.130650,4.928157,-0.003461,-0.011325,-0.014437,-0.008315,-0.001399
15/09/2004,2.040883,2.297686,3.395869,4.128747,4.921745,0.002981,0.001825,-0.000013,-0.001903,-0.006412
16/09/2004,2.047758,2.308354,3.404290,4.130073,4.913589,0.006875,0.010669,0.008421,0.001326,-0.008156
17/09/2004,2.050293,2.281755,3.360508,4.098976,4.893322,0.002536,-0.026599,-0.043782,-0.031097,-0.020267


In [7]:
len(df)


4918

In [8]:
nombre_de_lignes_avant =len(df)

df = df.dropna()

nombre_de_lignes_apres = len(df)

print(f"Nombre de lignes supprimées : {nombre_de_lignes_avant-nombre_de_lignes_apres}")
print(f"Nombre de lignes  : {len(df)}")

Nombre de lignes supprimées : 0
Nombre de lignes  : 4918


**Importation of yield curve Dollar + cleaning**

In [9]:
dftaux_Dollar = pd.read_csv(filepath_or_buffer = './Courbe de taux $.csv',sep = ',', parse_dates=['Date'], index_col=['Date']).sort_index(ascending=True)

/tmp/ipykernel_4849/1615737557.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dftaux_Dollar = pd.read_csv(filepath_or_buffer = './Courbe de taux $.csv',sep = ',', parse_dates=['Date'], index_col=['Date']).sort_index(ascending=True)


In [10]:
dftaux_Dollar.index = dftaux_Dollar.index.strftime(date_format)
dftaux_Dollar.head(10)

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,,
02/01/2001,NaN,NaN,5.87,NaN,5.58,5.11,4.87,4.82,4.76,4.97,4.92,5.46,5.35
03/01/2001,NaN,NaN,5.69,NaN,5.44,5.04,4.92,4.92,4.94,5.18,5.14,5.62,5.49
04/01/2001,NaN,NaN,5.37,NaN,5.20,4.82,4.77,4.78,4.82,5.07,5.03,5.56,5.44
05/01/2001,NaN,NaN,5.12,NaN,4.98,4.60,4.56,4.57,4.66,4.93,4.93,5.50,5.41
08/01/2001,NaN,NaN,5.19,NaN,5.03,4.61,4.54,4.55,4.65,4.94,4.94,5.52,5.42
09/01/2001,NaN,NaN,5.24,NaN,5.11,4.71,4.64,4.65,4.73,4.98,4.98,5.53,5.43
10/01/2001,NaN,NaN,5.29,NaN,5.16,4.82,4.76,4.78,4.83,5.09,5.10,5.60,5.49
11/01/2001,NaN,NaN,5.31,NaN,5.17,4.84,4.77,4.78,4.85,5.12,5.14,5.67,5.55
12/01/2001,NaN,NaN,5.33,NaN,5.24,4.96,4.90,4.91,4.97,5.24,5.25,5.74,5.63


In [11]:
dftaux_Dollar = dftaux_Dollar[['1 Mo', '3 Mo', '1 Yr', '5 Yr', '10 Yr']]
dftaux_Dollar.head(10)

,1 Mo,3 Mo,1 Yr,5 Yr,10 Yr
Date,,,,,
02/01/2001,NaN,5.87,5.11,4.76,4.92
03/01/2001,NaN,5.69,5.04,4.94,5.14
04/01/2001,NaN,5.37,4.82,4.82,5.03
05/01/2001,NaN,5.12,4.60,4.66,4.93
08/01/2001,NaN,5.19,4.61,4.65,4.94
09/01/2001,NaN,5.24,4.71,4.73,4.98
10/01/2001,NaN,5.29,4.82,4.83,5.10
11/01/2001,NaN,5.31,4.84,4.85,5.14
12/01/2001,NaN,5.33,4.96,4.97,5.25


In [12]:
nombre_lignes_avant = len(dftaux_Dollar)

for x in ['1 Mo', '3 Mo', '1 Yr', '5 Yr', '10 Yr']:
    dftaux_Dollar[f'Return_YC_Dollar_{x}'] = dftaux_Dollar[x] - dftaux_Dollar[x].shift(1)
dftaux_Dollar.dropna(inplace=True)

nombre_lignes_apres = len(dftaux_Dollar)
lignes_supprimees = nombre_lignes_avant - nombre_lignes_apres

print(f"Nombre de lignes supprimées : {lignes_supprimees}")
print(f"Nombre de lignes : {len(dftaux_Dollar)}")
dftaux_Dollar.head(10)

Nombre de lignes supprimées : 154
Nombre de lignes : 5100


,1 Mo,3 Mo,1 Yr,5 Yr,10 Yr,Return_YC_Dollar_1 Mo,Return_YC_Dollar_3 Mo,Return_YC_Dollar_1 Yr,Return_YC_Dollar_5 Yr,Return_YC_Dollar_10 Yr
Date,,,,,,,,,,
01/08/2001,3.65,3.53,3.56,4.62,5.11,-0.02,-0.01,0.03,0.05,0.04
02/08/2001,3.65,3.53,3.57,4.69,5.17,0.00,0.00,0.01,0.07,0.06
03/08/2001,3.63,3.52,3.57,4.72,5.20,-0.02,-0.01,0.00,0.03,0.03
06/08/2001,3.62,3.52,3.56,4.71,5.19,-0.01,0.00,-0.01,-0.01,-0.01
07/08/2001,3.63,3.52,3.56,4.72,5.20,0.01,0.00,0.00,0.01,0.01
08/08/2001,3.61,3.49,3.46,4.61,4.99,-0.02,-0.03,-0.10,-0.11,-0.21
09/08/2001,3.61,3.45,3.48,4.66,5.04,0.00,-0.04,0.02,0.05,0.05
10/08/2001,3.58,3.43,3.45,4.61,4.99,-0.03,-0.02,-0.03,-0.05,-0.05
13/08/2001,3.57,3.45,3.43,4.57,4.97,-0.01,0.02,-0.02,-0.04,-0.02


**Importation of stock indices from Yahoo Finance + Cleaning**

S&P 500


In [13]:
sp500_data = yf.download('^GSPC', start='2013-10-24', end='2023-10-24', progress=False)
sp500_data.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-10-24,1747.479980,1753.939941,1745.500000,1752.069946,1752.069946,3671700000
2013-10-25,1756.010010,1759.819946,1752.449951,1759.770020,1759.770020,3175720000
2013-10-28,1759.420044,1764.989990,1757.670044,1762.109985,1762.109985,3282300000
2013-10-29,1762.930054,1772.089966,1762.930054,1771.949951,1771.949951,3358460000
2013-10-30,1772.270020,1775.219971,1757.239990,1763.310059,1763.310059,3523040000
2013-10-31,1763.239990,1768.530029,1755.719971,1756.540039,1756.540039,3826530000
2013-11-01,1758.699951,1765.670044,1752.699951,1761.640015,1761.640015,3686290000
2013-11-04,1763.400024,1768.780029,1761.560059,1767.930054,1767.930054,3194870000
2013-11-05,1765.670044,1767.030029,1755.760010,1762.969971,1762.969971,3516680000


In [14]:
print(sp500_data.columns)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


In [15]:
sp500_data.reset_index(inplace=True)  # Réinitialise l'index pour obtenir la colonne "Date"
sp500_data = sp500_data[["Date", "Adj Close"]].rename(columns={'Adj Close': 'sp500'})

sp500_data['Date'] = pd.to_datetime(sp500_data['Date']).dt.strftime(date_format)

In [16]:
sp500_data.head(10)

,Date,sp500
0,24/10/2013,1752.069946
1,25/10/2013,1759.770020
2,28/10/2013,1762.109985
3,29/10/2013,1771.949951
4,30/10/2013,1763.310059
5,31/10/2013,1756.540039
6,01/11/2013,1761.640015
7,04/11/2013,1767.930054
8,05/11/2013,1762.969971
9,06/11/2013,1770.489990


Eurostoxx50

In [17]:
Eurostoxx50_data = yf.download('^STOXX50E', start='2013-10-24', end='2023-10-24', progress=False)
print(Eurostoxx50_data.head())
Eurostoxx50_data.size

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2013-10-24  3024.689941  3038.959961  3022.280029  3038.959961  3038.959961   
2013-10-25  3027.840088  3043.840088  3021.010010  3034.500000  3034.500000   
2013-10-28  3041.159912  3045.139893  3009.800049  3022.040039  3022.040039   
2013-10-29  3019.649902  3050.739990  3017.729980  3050.639893  3050.639893   
2013-10-30  3049.090088  3073.879883  3035.020020  3040.689941  3040.689941   

              Volume  
Date                  
2013-10-24  53819900  
2013-10-25  49404800  
2013-10-28  51393200  
2013-10-29  59850600  
2013-10-30  60923400  


15054

In [18]:
Eurostoxx50_data.reset_index(inplace=True)  # Réinitialise l'index pour obtenir la colonne "Date"
Eurostoxx50_data = Eurostoxx50_data[["Date", "Adj Close"]].rename(columns={'Adj Close': 'Eurostoxx50'})

Eurostoxx50_data['Date'] = pd.to_datetime(Eurostoxx50_data['Date']).dt.strftime(date_format)

In [19]:
Eurostoxx50_data.head(10)

,Date,Eurostoxx50
0,24/10/2013,3038.959961
1,25/10/2013,3034.500000
2,28/10/2013,3022.040039
3,29/10/2013,3050.639893
4,30/10/2013,3040.689941
5,31/10/2013,3067.949951
6,01/11/2013,3052.139893
7,04/11/2013,3061.179932
8,05/11/2013,3035.919922
9,06/11/2013,3056.399902


**Euro/Dollar exchange rate importation + cleaning**

In [20]:
dftaux = pd.read_csv(filepath_or_buffer = './Taux de change euro dollar.csv',sep = ',')[['_TIME_PERIOD', '_OBS_VALUE']].rename(columns={'_TIME_PERIOD': 'Date', '_OBS_VALUE': 'TauxdeChange'})
dftaux['Date'] = pd.to_datetime(dftaux['Date']).dt.strftime(date_format)
dftaux.head(10)

,Date,TauxdeChange
0,04/01/1999,1.1789
1,05/01/1999,1.1790
2,06/01/1999,1.1743
3,07/01/1999,1.1632
4,08/01/1999,1.1659
5,11/01/1999,1.1569
6,12/01/1999,1.1520
7,13/01/1999,1.1744
8,14/01/1999,1.1653
9,15/01/1999,1.1626


In [21]:
dftaux = pd.merge(dftaux, sp500_data, on='Date', how='inner')
dftaux = pd.merge(dftaux, Eurostoxx50_data, on='Date', how='inner')
dftaux.set_index('Date', inplace=True)

In [22]:
dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50
Date,,,
24/10/2013,1.3805,1752.069946,3038.959961
25/10/2013,1.3777,1759.770020,3034.500000
28/10/2013,1.3784,1762.109985,3022.040039
29/10/2013,1.3768,1771.949951,3050.639893
30/10/2013,1.3755,1763.310059,3040.689941
31/10/2013,1.3641,1756.540039,3067.949951
01/11/2013,1.3505,1761.640015,3052.139893
04/11/2013,1.3506,1767.930054,3061.179932
05/11/2013,1.3494,1762.969971,3035.919922


**Yields calculation**

In [23]:

dftaux['Log_Return_TauxdeChange'] = np.log(dftaux['TauxdeChange'] / dftaux['TauxdeChange'].shift(1))

In [24]:
dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50,Log_Return_TauxdeChange
Date,,,,
24/10/2013,1.3805,1752.069946,3038.959961,NaN
25/10/2013,1.3777,1759.770020,3034.500000,-0.002030
28/10/2013,1.3784,1762.109985,3022.040039,0.000508
29/10/2013,1.3768,1771.949951,3050.639893,-0.001161
30/10/2013,1.3755,1763.310059,3040.689941,-0.000945
31/10/2013,1.3641,1756.540039,3067.949951,-0.008322
01/11/2013,1.3505,1761.640015,3052.139893,-0.010020
04/11/2013,1.3506,1767.930054,3061.179932,0.000074
05/11/2013,1.3494,1762.969971,3035.919922,-0.000889


In [25]:

dftaux['Log_Return_SP500'] = np.log(dftaux['sp500'] / dftaux['sp500'].shift(1))


In [26]:
nombre_de_lignes_avant =len(dftaux)

dftaux = dftaux.dropna()

nombre_de_lignes_apres = len(dftaux)

print(f"Nombre de lignes supprimées : {nombre_de_lignes_avant-nombre_de_lignes_apres}")
print(f"Nombre de lignes  : {len(dftaux)}")


Nombre de lignes supprimées : 1
Nombre de lignes  : 2448


In [27]:
dftaux['Log_Return_Eurostoxx50'] = np.log(dftaux['Eurostoxx50'] / dftaux['Eurostoxx50'].shift(1))

dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50,Log_Return_TauxdeChange,Log_Return_SP500,Log_Return_Eurostoxx50
Date,,,,,,
25/10/2013,1.3777,1759.770020,3034.500000,-0.002030,0.004385,NaN
28/10/2013,1.3784,1762.109985,3022.040039,0.000508,0.001329,-0.004115
29/10/2013,1.3768,1771.949951,3050.639893,-0.001161,0.005569,0.009419
30/10/2013,1.3755,1763.310059,3040.689941,-0.000945,-0.004888,-0.003267
31/10/2013,1.3641,1756.540039,3067.949951,-0.008322,-0.003847,0.008925
01/11/2013,1.3505,1761.640015,3052.139893,-0.010020,0.002899,-0.005167
04/11/2013,1.3506,1767.930054,3061.179932,0.000074,0.003564,0.002957
05/11/2013,1.3494,1762.969971,3035.919922,-0.000889,-0.002810,-0.008286
06/11/2013,1.3517,1770.489990,3056.399902,0.001703,0.004256,0.006723


In [28]:
nombre_de_lignes_avant =len(dftaux)

dftaux = dftaux.dropna()

nombre_de_lignes_apres = len(dftaux)

print(f"Nombre de lignes supprimées : {nombre_de_lignes_avant-nombre_de_lignes_apres}")
print(f"Nombre de lignes  : {len(dftaux)}")

Nombre de lignes supprimées : 1
Nombre de lignes  : 2447


In [29]:
dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50,Log_Return_TauxdeChange,Log_Return_SP500,Log_Return_Eurostoxx50
Date,,,,,,
28/10/2013,1.3784,1762.109985,3022.040039,0.000508,0.001329,-0.004115
29/10/2013,1.3768,1771.949951,3050.639893,-0.001161,0.005569,0.009419
30/10/2013,1.3755,1763.310059,3040.689941,-0.000945,-0.004888,-0.003267
31/10/2013,1.3641,1756.540039,3067.949951,-0.008322,-0.003847,0.008925
01/11/2013,1.3505,1761.640015,3052.139893,-0.010020,0.002899,-0.005167
04/11/2013,1.3506,1767.930054,3061.179932,0.000074,0.003564,0.002957
05/11/2013,1.3494,1762.969971,3035.919922,-0.000889,-0.002810,-0.008286
06/11/2013,1.3517,1770.489990,3056.399902,0.001703,0.004256,0.006723
07/11/2013,1.3365,1747.150024,3042.979980,-0.011309,-0.013270,-0.004400


In [30]:
#min , max , valeur moyenne, nb de valeurs manquantes, 
#faire statistiques descriptives
#Pour check actions: on calcul les rendements des indices (autour de 10% max)

#Pour les courbes de taux : 
#en euro taux descendu jusqu'à -O.5% et ajd max autour de 4%
#en USD min autour de 0 et max autour de 5 

#Gérer les outliers (données biaisées)

dftaux.describe()

,TauxdeChange,sp500,Eurostoxx50,Log_Return_TauxdeChange,Log_Return_SP500,Log_Return_Eurostoxx50
count,2447.000000,2447.000000,2447.000000,2447.000000,2447.000000,2447.000000
mean,1.147984,2960.728553,3499.893612,-0.000107,0.000357,0.000117
std,0.085797,892.103776,404.557927,0.005079,0.011300,0.012545
min,0.956500,1741.890015,2385.820068,-0.036820,-0.127652,-0.132405
25%,1.093450,2107.589966,3194.085083,-0.002799,-0.003890,-0.005517
50%,1.128500,2770.370117,3451.969971,-0.000077,0.000575,0.000574
75%,1.182000,3844.950073,3706.714966,0.002683,0.005622,0.006225
max,1.395300,4796.560059,4471.310059,0.034946,0.089683,0.088343


In [31]:
df.describe()

KEY,3 Mo,1 Yr,5 Yr,10 Yr,30 Yr,Return_YC_Euro_3 Mo,Return_YC_Euro_1 Yr,Return_YC_Euro_5 Yr,Return_YC_Euro_10 Yr,Return_YC_Euro_30 Yr
count,4918.000000,4918.000000,4918.000000,4918.000000,4918.000000,4918.000000,4918.000000,4918.000000,4918.000000,4918.000000
mean,0.708769,0.795462,1.316927,1.928515,2.454659,0.000350,0.000212,-0.000195,-0.000329,-0.000451
std,1.553148,1.613033,1.613202,1.624925,1.560312,0.027669,0.027582,0.040817,0.041092,0.048277
min,-0.929979,-0.912983,-0.996219,-0.815215,-0.432375,-0.939686,-0.396571,-0.312484,-0.264642,-0.564029
25%,-0.602827,-0.631093,-0.263019,0.398840,1.156612,-0.007825,-0.009190,-0.020377,-0.023918,-0.026697
50%,0.052046,0.091548,0.934054,1.991801,2.532290,0.000437,0.000059,-0.000589,-0.000364,-0.000774
75%,2.040099,2.122741,2.727654,3.526134,3.955197,0.008271,0.009956,0.020002,0.022800,0.023930
max,4.325459,4.539553,4.730363,4.776331,5.175029,0.272439,0.202086,0.241246,0.218434,0.310749


In [32]:
dftaux_Dollar.describe()

,1 Mo,3 Mo,1 Yr,5 Yr,10 Yr,Return_YC_Dollar_1 Mo,Return_YC_Dollar_3 Mo,Return_YC_Dollar_1 Yr,Return_YC_Dollar_5 Yr,Return_YC_Dollar_10 Yr
count,5100.00000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000
mean,1.19039,1.241704,1.447041,2.354975,3.033139,-0.000694,-0.000676,-0.000612,-0.000639,-0.000678
std,1.45444,1.478164,1.483270,1.290397,1.199166,0.061100,0.044612,0.036624,0.058247,0.056534
min,0.00000,0.000000,0.040000,0.190000,0.520000,-1.050000,-0.810000,-0.500000,-0.460000,-0.510000
25%,0.06000,0.080000,0.190000,1.390000,2.050000,-0.010000,-0.010000,-0.010000,-0.030000,-0.030000
50%,0.51000,0.650000,1.010000,2.050000,2.865000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.80000,1.870000,2.250000,3.280000,4.110000,0.010000,0.010000,0.010000,0.030000,0.030000
max,5.27000,5.190000,5.300000,5.230000,5.440000,0.860000,0.760000,0.520000,0.340000,0.290000


In [33]:
#Detection des outliers ici méthode de l'ecart interquartile (IQR)

In [34]:

for x in ['1 Mo','3 Mo','1 Yr','5 Yr','10 Yr','Return_YC_Dollar_1 Mo','Return_YC_Dollar_3 Mo','Return_YC_Dollar_1 Yr','Return_YC_Dollar_5 Yr','Return_YC_Dollar_10 Yr']:    
    Q1 = dftaux_Dollar[x].quantile(0.25)
    Q3 = dftaux_Dollar[x].quantile(0.75)
    IQR = Q3 - Q1 #étendue de la moitié centrale des données
    outliers = dftaux_Dollar[(dftaux_Dollar[x] < (Q1 - 1.5 * IQR)) | (dftaux_Dollar[x] > (Q3 + 1.5 * IQR))]
    print(f"Nombre d'outliers pour {x} : {len(outliers)}")

Nombre d'outliers pour 1 Mo : 372
Nombre d'outliers pour 3 Mo : 375
Nombre d'outliers pour 1 Yr : 0
Nombre d'outliers pour 5 Yr : 0
Nombre d'outliers pour 10 Yr : 0
Nombre d'outliers pour Return_YC_Dollar_1 Mo : 650
Nombre d'outliers pour Return_YC_Dollar_3 Mo : 430
Nombre d'outliers pour Return_YC_Dollar_1 Yr : 593
Nombre d'outliers pour Return_YC_Dollar_5 Yr : 221
Nombre d'outliers pour Return_YC_Dollar_10 Yr : 188


In [35]:
for x in ['TauxdeChange', 'sp500', 'Eurostoxx50', 'Log_Return_SP500','Log_Return_Eurostoxx50','Log_Return_TauxdeChange']:    
    Q1 = dftaux[x].quantile(0.25)
    Q3 = dftaux[x].quantile(0.75)
    IQR = Q3 - Q1 #étendue de la moitié centrale des données
    outliers = dftaux[(dftaux[x] < (Q1 - 1.5 * IQR)) | (dftaux[x] > (Q3 + 1.5 * IQR))]
    print(f"Nombre d'outliers pour {x} : {len(outliers)}")

Nombre d'outliers pour TauxdeChange : 205
Nombre d'outliers pour sp500 : 0
Nombre d'outliers pour Eurostoxx50 : 1
Nombre d'outliers pour Log_Return_SP500 : 171
Nombre d'outliers pour Log_Return_Eurostoxx50 : 148
Nombre d'outliers pour Log_Return_TauxdeChange : 97


In [36]:
for x in ['3 Mo', '1 Yr', '5 Yr', '10 Yr']:    
    Q1 = df[x].quantile(0.25)
    Q3 = df[x].quantile(0.75)
    IQR = Q3 - Q1 #étendue de la moitié centrale des données
    outliers = df[(df[x] < (Q1 - 1.5 * IQR)) | (df[x] > (Q3 + 1.5 * IQR))]
    print(f"Nombre d'outliers pour {x} : {len(outliers)}")

Nombre d'outliers pour 3 Mo : 0
Nombre d'outliers pour 1 Yr : 0
Nombre d'outliers pour 5 Yr : 0
Nombre d'outliers pour 10 Yr : 0


In [37]:
#Detection des outliers ici méthode Z-score


In [38]:

for x in ['1 Mo','3 Mo','1 Yr','5 Yr','10 Yr','Return_YC_Dollar_1 Mo','Return_YC_Dollar_3 Mo','Return_YC_Dollar_1 Yr','Return_YC_Dollar_5 Yr','Return_YC_Dollar_10 Yr']:    
    z = np.abs(stats.zscore(dftaux_Dollar[x]))
    outliers = dftaux_Dollar[z > 3]
    print(f"Nombre d'outliers pour {x} : {len(outliers)}")


Nombre d'outliers pour 1 Mo : 0
Nombre d'outliers pour 3 Mo : 0
Nombre d'outliers pour 1 Yr : 0
Nombre d'outliers pour 5 Yr : 0
Nombre d'outliers pour 10 Yr : 0
Nombre d'outliers pour Return_YC_Dollar_1 Mo : 93
Nombre d'outliers pour Return_YC_Dollar_3 Mo : 83
Nombre d'outliers pour Return_YC_Dollar_1 Yr : 85
Nombre d'outliers pour Return_YC_Dollar_5 Yr : 71
Nombre d'outliers pour Return_YC_Dollar_10 Yr : 57


In [39]:
for x in ['TauxdeChange', 'sp500', 'Eurostoxx50', 'Log_Return_SP500','Log_Return_Eurostoxx50','Log_Return_TauxdeChange']:    
    z = np.abs(stats.zscore(dftaux[x]))
    outliers = dftaux[z > 3]
    print(f"Nombre d'outliers pour {x} : {len(outliers)}")

Nombre d'outliers pour TauxdeChange : 0
Nombre d'outliers pour sp500 : 0
Nombre d'outliers pour Eurostoxx50 : 0
Nombre d'outliers pour Log_Return_SP500 : 35
Nombre d'outliers pour Log_Return_Eurostoxx50 : 34
Nombre d'outliers pour Log_Return_TauxdeChange : 29


In [40]:
for x in ['3 Mo', '1 Yr', '5 Yr', '10 Yr']:    
    z = np.abs(stats.zscore(df[x]))
    outliers = df[z > 3]
    print(f"Nombre d'outliers pour {x} : {len(outliers)}")

Nombre d'outliers pour 3 Mo : 0
Nombre d'outliers pour 1 Yr : 0
Nombre d'outliers pour 5 Yr : 0
Nombre d'outliers pour 10 Yr : 0


***Generating scenarios with bootstrapping method***

Scenarios constants definition

In [41]:
nb_scenarios = 1000

Scenarios Actions

In [45]:
import pandas as pd
import numpy as np


np.random.seed(1)

# Nombre de scénarios à générer
nb_scenarios = 1000

# Sélection aléatoire des indices dans les données historiques pour chaque scénario
indices_aleatoires = np.random.choice(dftaux.index, nb_scenarios, replace=True)
# Récupérer les rendements correspondants pour chaque indice sélectionné
scenarios = [dftaux.loc[indices].values for indices in indices_aleatoires]


# Convertir en DataFrame et ajouter la valeur initiale
dftaux_scenarios = pd.DataFrame(scenarios)
dftaux_scenarios = pd.DataFrame(scenarios, columns=dftaux.columns)
dftaux_scenarios.index = indices_aleatoires




dftaux_scenarios

,TauxdeChange,sp500,Eurostoxx50,Log_Return_TauxdeChange,Log_Return_SP500,Log_Return_Eurostoxx50
06/05/2019,1.1199,2932.469971,3462.949951,0.003937,-0.004481,-0.011350
04/06/2018,1.1737,2746.870117,3469.570068,0.005810,0.004470,0.004631
22/01/2021,1.2158,3841.469971,3602.409912,0.000000,-0.003015,-0.004415
18/08/2022,1.0178,4283.740234,3777.379883,0.001376,0.002267,0.005660
10/07/2018,1.1713,2793.840088,3473.310059,-0.006468,0.003467,0.003712
...,...,...,...,...,...,...
03/08/2022,1.0194,4155.169922,3732.540039,-0.002939,0.015517,0.012919
11/01/2018,1.2017,2767.560059,3595.239990,0.002083,0.007009,-0.004050
11/08/2021,1.1718,4442.410156,4206.330078,-0.000341,0.001275,0.004410
21/12/2018,1.1414,2416.620117,3000.610107,-0.003236,-0.020803,0.000183


In [54]:
#Calcul des statistiques des prix dans les scénarios (moyenne, mon, max, écart-type, centiles 5% et 95%)

moyenne_prix = dftaux_scenarios.mean()
minimum_prix = dftaux_scenarios.min()
maximum_prix = dftaux_scenarios.max()
ecart_type_prix = dftaux_scenarios.std()
centile_5_prix = dftaux_scenarios.quantile(0.05)
centile_95_prix = dftaux_scenarios.quantile(0.95)

print("Moyenne des prix :\n", moyenne_prix)
print("\nMinimum des prix :\n", minimum_prix)
print("\nMaximum des prix :\n", maximum_prix)
print("\nÉcart-type des prix :\n", ecart_type_prix)
print("\nCentile 5 des prix :\n", centile_5_prix)
print("\nCentile 95 des prix :\n", centile_95_prix)

Moyenne des prix :
 TauxdeChange                  1.149281
sp500                      2995.736633
Eurostoxx50                3517.900891
Log_Return_TauxdeChange      -0.000267
Log_Return_SP500              0.000647
Log_Return_Eurostoxx50       -0.000255
dtype: float64

Minimum des prix :
 TauxdeChange                  0.970600
sp500                      1747.150024
Eurostoxx50                2385.820068
Log_Return_TauxdeChange      -0.021549
Log_Return_SP500             -0.099945
Log_Return_Eurostoxx50       -0.132405
dtype: float64

Maximum des prix :
 TauxdeChange                  1.395300
sp500                      4725.790039
Eurostoxx50                4471.310059
Log_Return_TauxdeChange       0.024664
Log_Return_SP500              0.089683
Log_Return_Eurostoxx50        0.088343
dtype: float64

Écart-type des prix :
 TauxdeChange                 0.087177
sp500                      926.110180
Eurostoxx50                409.768810
Log_Return_TauxdeChange      0.004989
Log_Return_SP50

Scenarios Monnaies

In [56]:
import pandas as pd
import numpy as np


np.random.seed(1)

# Nombre de scénarios à générer
nb_scenarios = 1000

# Sélection aléatoire des indices dans les données historiques pour chaque scénario
indices_aleatoires = np.random.choice(df.index, nb_scenarios, replace=True)
# Récupérer les rendements correspondants pour chaque indice sélectionné
scenarios = [df.loc[indices].values for indices in indices_aleatoires]


# Convertir en DataFrame et ajouter la valeur initiale
df_scenarios = pd.DataFrame(scenarios)
df_scenarios = pd.DataFrame(scenarios, columns=df.columns)
df_scenarios.index = indices_aleatoires




df_scenarios

KEY,3 Mo,1 Yr,5 Yr,10 Yr,30 Yr,Return_YC_Euro_3 Mo,Return_YC_Euro_1 Yr,Return_YC_Euro_5 Yr,Return_YC_Euro_10 Yr,Return_YC_Euro_30 Yr
04/08/2005,2.062558,2.122855,2.754356,3.347987,3.970478,0.001798,0.002006,-0.004960,-0.010635,-0.014405
01/04/2020,-0.647068,-0.651190,-0.620679,-0.379292,0.064820,0.053798,0.043631,0.044597,0.034929,0.005284
18/03/2008,3.834001,3.513868,3.433113,4.031010,4.898653,0.015410,0.065371,0.079395,0.050921,0.062277
23/06/2015,-0.246280,-0.239920,0.255284,1.027030,1.853275,-0.009161,-0.007443,-0.010072,0.002300,0.024727
28/12/2015,-0.420946,-0.379299,0.011282,0.736978,1.636110,0.005331,0.002950,-0.006215,-0.016350,-0.033942
...,...,...,...,...,...,...,...,...,...,...
05/12/2022,1.427477,2.009291,1.887548,1.917059,1.729366,-0.016313,0.024227,0.021979,0.012094,-0.032279
18/05/2012,0.068899,0.086851,1.132098,2.223842,2.745716,0.000847,0.010685,-0.000879,-0.015112,0.006370
07/09/2009,0.376145,0.631102,2.637216,3.720372,4.329131,-0.015001,-0.033805,-0.004349,0.019869,0.055060
12/04/2023,2.817753,2.883133,2.389260,2.435997,2.418316,0.014324,0.043056,0.055544,0.051488,0.079460


In [57]:
#Calcul des statistiques des prix dans les scénarios (moyenne, mon, max, écart-type, centiles 5% et 95%)

moyenne_prix = df_scenarios.mean()
minimum_prix = df_scenarios.min()
maximum_prix = df_scenarios.max()
ecart_type_prix = df_scenarios.std()
centile_5_prix = df_scenarios.quantile(0.05)
centile_95_prix = df_scenarios.quantile(0.95)

print("Moyenne des prix :\n", moyenne_prix)
print("\nMinimum des prix :\n", minimum_prix)
print("\nMaximum des prix :\n", maximum_prix)
print("\nÉcart-type des prix :\n", ecart_type_prix)
print("\nCentile 5 des prix :\n", centile_5_prix)
print("\nCentile 95 des prix :\n", centile_95_prix)

Moyenne des prix :
 KEY
3 Mo                    0.705861
1 Yr                    0.784375
5 Yr                    1.290159
10 Yr                   1.902887
30 Yr                   2.433089
Return_YC_Euro_3 Mo     0.001017
Return_YC_Euro_1 Yr     0.000123
Return_YC_Euro_5 Yr     0.000606
Return_YC_Euro_10 Yr    0.000318
Return_YC_Euro_30 Yr   -0.000689
dtype: float64

Minimum des prix :
 KEY
3 Mo                   -0.907309
1 Yr                   -0.885551
5 Yr                   -0.918779
10 Yr                  -0.702523
30 Yr                  -0.414038
Return_YC_Euro_3 Mo    -0.122931
Return_YC_Euro_1 Yr    -0.263676
Return_YC_Euro_5 Yr    -0.182562
Return_YC_Euro_10 Yr   -0.184104
Return_YC_Euro_30 Yr   -0.252029
dtype: float64

Maximum des prix :
 KEY
3 Mo                    4.301273
1 Yr                    4.539553
5 Yr                    4.724605
10 Yr                   4.776331
30 Yr                   4.986063
Return_YC_Euro_3 Mo     0.210542
Return_YC_Euro_1 Yr     0.152906
Retur

In [58]:
import pandas as pd
import numpy as np


np.random.seed(1)

# Nombre de scénarios à générer
nb_scenarios = 1000

# Sélection aléatoire des indices dans les données historiques pour chaque scénario
indices_aleatoires = np.random.choice(dftaux_Dollar.index, nb_scenarios, replace=True)
# Récupérer les rendements correspondants pour chaque indice sélectionné
scenarios = [dftaux_Dollar.loc[indices].values for indices in indices_aleatoires]


# Convertir en DataFrame et ajouter la valeur initiale
dftaux_Dollar_scenarios = pd.DataFrame(scenarios)
dftaux_Dollar_scenarios = pd.DataFrame(scenarios, columns=dftaux_Dollar.columns)
dftaux_Dollar_scenarios.index = indices_aleatoires




dftaux_Dollar_scenarios

,1 Mo,3 Mo,1 Yr,5 Yr,10 Yr,Return_YC_Dollar_1 Mo,Return_YC_Dollar_3 Mo,Return_YC_Dollar_1 Yr,Return_YC_Dollar_5 Yr,Return_YC_Dollar_10 Yr
15/07/2002,1.71,1.72,1.97,3.85,4.66,0.00,0.01,0.00,0.03,0.03
12/07/2017,0.94,1.05,1.21,1.88,2.33,-0.03,0.00,0.01,-0.04,-0.04
21/03/2005,2.72,2.85,3.33,4.18,4.53,0.02,0.04,0.01,0.00,0.02
27/08/2012,0.10,0.11,0.18,0.70,1.65,0.00,0.01,-0.01,-0.02,-0.03
12/03/2013,0.10,0.10,0.15,0.88,2.03,0.01,0.00,0.00,-0.02,-0.04
...,...,...,...,...,...,...,...,...,...,...
06/04/2007,5.10,5.05,4.98,4.67,4.76,0.00,0.01,0.05,0.10,0.08
13/10/2005,3.50,3.75,4.14,4.32,4.48,0.01,0.05,0.00,0.00,0.03
24/08/2017,0.98,1.02,1.23,1.78,2.19,0.00,0.02,0.01,0.02,0.02
08/08/2008,1.63,1.70,2.19,3.21,3.94,0.10,0.04,0.02,0.05,0.02


In [60]:
#Calcul des statistiques des prix dans les scénarios (moyenne, mon, max, écart-type, centiles 5% et 95%)

moyenne_prix = dftaux_Dollar_scenarios.mean()
minimum_prix = dftaux_Dollar_scenarios.min()
maximum_prix = dftaux_Dollar_scenarios.max()
ecart_type_prix = dftaux_Dollar_scenarios.std()
centile_5_prix = dftaux_Dollar_scenarios.quantile(0.05)
centile_95_prix = dftaux_Dollar_scenarios.quantile(0.95)

print("Moyenne des prix :\n", moyenne_prix)
print("\nMinimum des prix :\n", minimum_prix)
print("\nMaximum des prix :\n", maximum_prix)
print("\nÉcart-type des prix :\n", ecart_type_prix)
print("\nCentile 5 des prix :\n", centile_5_prix)
print("\nCentile 95 des prix :\n", centile_95_prix)

Moyenne des prix :
 1 Mo                      1.10967
3 Mo                      1.16084
1 Yr                      1.37074
5 Yr                      2.30555
10 Yr                     3.00622
Return_YC_Dollar_1 Mo    -0.00229
Return_YC_Dollar_3 Mo    -0.00052
Return_YC_Dollar_1 Yr     0.00116
Return_YC_Dollar_5 Yr     0.00407
Return_YC_Dollar_10 Yr    0.00334
dtype: float64

Minimum des prix :
 1 Mo                      0.00
3 Mo                      0.00
1 Yr                      0.04
5 Yr                      0.25
10 Yr                     0.58
Return_YC_Dollar_1 Mo    -1.05
Return_YC_Dollar_3 Mo    -0.51
Return_YC_Dollar_1 Yr    -0.40
Return_YC_Dollar_5 Yr    -0.23
Return_YC_Dollar_10 Yr   -0.25
dtype: float64

Maximum des prix :
 1 Mo                      5.23
3 Mo                      5.19
1 Yr                      5.28
5 Yr                      5.18
10 Yr                     5.44
Return_YC_Dollar_1 Mo     0.53
Return_YC_Dollar_3 Mo     0.46
Return_YC_Dollar_1 Yr     0.21
Return_YC_

In [46]:
#scenarios_bootstrap_Eurostoxx50 = []
#for i in range(nb_scenarios):
    #new_scenario = np.random.choice(dftaux['Log_Return_Eurostoxx50'] , len(dftaux) , replace=True)    
    #scenarios_bootstrap_Eurostoxx50.append(new_scenario)
#print(scenarios_bootstrap_Eurostoxx50)

In [47]:
#scenarios_totaux_Eurostoxx50 = sum(scenarios_bootstrap_Eurostoxx50)
#print(scenarios_totaux_Eurostoxx50)

Exchange rates scenarios 

In [48]:
#scenarios_bootstrap_TauxdeChange = []
#for i in range(nb_scenarios):
    #new_scenario = np.random.choice(dftaux['Log_Return_TauxdeChange'] , len(dftaux) , replace=True)    
    #scenarios_bootstrap_TauxdeChange.append(new_scenario)
#print(scenarios_bootstrap_TauxdeChange)

In [49]:
#scenarios_totaux_TauxdeChange = sum(scenarios_bootstrap_TauxdeChange)
#print(scenarios_totaux_TauxdeChange)

SP500 scenarios

In [50]:
#scenarios_bootstrap_SP500 = []
#for i in range(nb_scenarios):
    #new_scenario = np.random.choice(dftaux['Log_Return_SP500'] , len(dftaux) , replace=True)    
    #scenarios_bootstrap_SP500.append(new_scenario)
#print(scenarios_bootstrap_SP500)

In [51]:
#scenarios_totaux_SP500 = sum(scenarios_bootstrap_SP500)
#print(scenarios_totaux_SP500)

***Swap pricing***

Uploading the XML file

In [ ]:
xml_data = """
<instrumentlist xmlns="http://www.statpro.net/xml/structure/1.0"><http://www.statpro.net/xml/structure/1.0%22%3e>
  <!-- plain swap -->
  <instrument>
    <code>TEST0183</code>
    <codetype>test-dp</codetype>
    <description>Sample interest-rate swap</description>
    <currency>EUR</currency>
    <swap>
      <counterpartycode>Foo</counterpartycode>
      <fixedrateleg>
        <side>Pay</side>
        <startdate>2008-03-18</startdate>
        <maturitydate>2018-03-17</maturitydate>
        <settlementdays>2</settlementdays>
        <calendar>TARGET</calendar>
        <rollingconvention>Following</rollingconvention>
        <frequency>1</frequency>
        <adjusted>false</adjusted>
        <daycount>30/360</daycount>
        <notionals>100.0</notionals>
        <couponrates>0.04</couponrates>
      </fixedrateleg>
      <floatingrateleg>
        <side>Receive</side>
        <startdate>2008-03-18</startdate>
        <maturitydate>2018-03-17</maturitydate>
        <settlementdays>2</settlementdays>
        <calendar>TARGET</calendar>
        <rollingconvention>ModifiedFollowing</rollingconvention>
        <frequency>2</frequency>
        <adjusted>true</adjusted>
        <daycount>Act/360</daycount>
        <notionals>100.0</notionals>
        <referenceindex>EUR-Euribor-6-m</referenceindex>
        <fixingdays>2</fixingdays>
        <fixingtype>Advance</fixingtype>
        <spreads>0.002</spreads>
      </floatingrateleg>
    </swap>
  </instrument>
</instrumentlist>
"""

SWAP constants definition

In [ ]:
notional = 100
fixed_rate = 0.04
frequency = 2
maturity = 10
startdate = datetime.strptime("18/03/2008", date_format)
maturitydate = datetime.strptime("17/03/2018", date_format)
market_price = 0  # à remplacer par le prix actuel du swap

In [ ]:
# Courbe des taux 
rate_curve = df[['10 Yr']]

print(rate_curve.iloc[6].item())

list_rate_curve = list(rate_curve["10 Yr"])

4.130649993338003


Functions definition

In [ ]:
# Fonction pour calculer les cash flows de la jambe fixe
def calculate_fixed_leg_coupon(notional, fixed_rate, frequency, maturity):
    fixed_leg_cash_flows = [(fixed_rate / frequency) * notional] * (maturity * frequency)
    return fixed_leg_cash_flows


# Fonction pour extraire l'année d'une date dans un dataframe
def extract_year_index(i, df=rate_curve, date_format=date_format):
    return datetime.strptime(df.iloc[i].name, date_format).year


# Fonction pour calculer les taux forward
def calculate_forward_rates(rate_curve, startdate):
    list_rate_curve = list(rate_curve["10 Yr"])
    forward_rates = [((((1 + list_rate_curve[i]) ** (extract_year_index(i) - startdate.year)) / ((1 + list_rate_curve[i - 1]) ** (extract_year_index(i - 1) - startdate.year))) ** (extract_year_index(i) - extract_year_index(i - 1))) - 1 for i in range(1, len(list_rate_curve))]
    return forward_rates


# Fonction pour calculer les cash flows de la jambe variable 
def calculate_variable_leg_coupon(forward_rates, notional, frequency, maturity):
    variable_leg_cash_flows = [(rate / frequency) * notional for rate in forward_rates]
    return variable_leg_cash_flows


# Fonction pour calculer le prix du swap
def calculate_swap_price(rate_curve, notional, fixed_rate, frequency, maturity, spread):
    fixed_leg_cash_flows = calculate_fixed_leg_coupon(notional, fixed_rate, frequency, maturity)
    forward_rates = calculate_forward_rates(rate_curve, startdate)
    variable_leg_cash_flows = calculate_variable_leg_coupon(forward_rates, notional, frequency, maturity)

   # Actualisation des cash flows avec le taux d'intérêt et le spread
    list_rate_curve = list(rate_curve["10 Yr"])
    discounted_fixed_leg = np.sum([cf / (1 + list_rate_curve[t] + spread / frequency) ** (t / frequency) for t, cf in enumerate(fixed_leg_cash_flows, start=1)])
    discounted_variable_leg = np.sum([cf / (1 + list_rate_curve[t]) ** (t / frequency) for t, cf in enumerate(variable_leg_cash_flows, start=1)])

    swap_price = discounted_fixed_leg - discounted_variable_leg
    return swap_price


# Fonction objectif pour la calibration du spread
def objective_function(spread, rate_curve, notional, fixed_rate, frequency, maturity, market_price):
    calculated_price = calculate_swap_price(rate_curve, notional, fixed_rate, frequency, maturity, spread)
    return (calculated_price - market_price) ** 2

In [ ]:
forward_rates = calculate_forward_rates(rate_curve, startdate)
print(len(forward_rates))

variable_leg_cash_flows = calculate_variable_leg_coupon(forward_rates, notional, frequency, maturity)
print(len(variable_leg_cash_flows))

print(list(enumerate(variable_leg_cash_flows, start=1)))

print()

for t, cf in enumerate(variable_leg_cash_flows, start=1):
    print(cf / (1 + list_rate_curve[t-1]) ** (t / frequency))


[cf / (1 + list_rate_curve[t-1]) ** (t / frequency) for t, cf in enumerate(variable_leg_cash_flows, start=1)]

4910
4910
[(1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (63, 0.0), (64, 0.0), (65, 0.0), (66, 0.0), (67, 0.0), (68, 0.0), (69, 0.0), (70, 0.0), (71, 0.0), (72, 0.0), (73, 0.0), (74, 0.0), (75, 0.0), (76, 0.0), (77, 0.0), (78, 0.0), (79, 0.0), (80, 0.0), (81, 0.0), (82, 0.0), (83, 0.0), (84, 0.0), (85, 191.92513238148612), (86, 0.0), (87, 0.0), (88, 0.0), (89, 0.0), (90,

OverflowError: (34, 'Numerical result out of range')

In [ ]:
# Initialisation du spread
initial_spread = 0.002

# Calibration du spread
calibrated_spread = optimize.minimize(objective_function, initial_spread, args=(rate_curve, notional, fixed_rate, frequency, maturity, market_price))
print(calibrated_spread)

# Calcul du prix du swap avec le spread calibré
final_price = calculate_swap_price(rate_curve, notional, fixed_rate, frequency, maturity, calibrated_spread)

print("Spread calibré:", calibrated_spread)
print("Prix du swap avec spread calibré:", final_price)

TypeError: Cannot index by location index with a non-integer key

***Questions***

- Les forward_rates calculés sont non-nuls pour seulement 19/4911 valeurs. Normal ?
- Idem pour les cash flows du variable leg.
- Peut-être une erreur sur le calcul du prix du swap (on obtient des valeurs infiniment proches de zéro qui ne peuvent pas être calculées)